In [11]:
import numpy as np
import scipy.special 
import matplotlib.pyplot as plt
import scipy.ndimage
%matplotlib inline

In [7]:
class neuralNetwork:
    
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate): #initialise
        
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        self.lr = learningrate
        
        self.wih = np.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))
        
        self.activation_function = lambda x: scipy.special.expit(x)
        
        self.inverse_activation_function = lambda x: scipy.special.logit(x)
        
        
    def train(self, inputs_list, targets_list): #train the NN
        
        inputs = np.array(inputs_list, ndmin = 2).T
        
        targets = np.array(targets_list, ndmin = 2).T
        
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        output_errors = targets - final_outputs
        
        hidden_errors = np.dot(self.who.T, output_errors)
        
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
        
        
    
    def query(self, input_list): #query the NN
        
        inputs = np.array(input_list, ndmin = 2).T #input list to 2d array
        
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
        

In [8]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.1

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [9]:
# load the mnist training data CSV file into a list
training_data_file = open("mnist_dataset/mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [17]:
epochs = 2 #change to 2

for e in range(epochs):

    for record in training_data_list:
            # split the record by the ',' commas
            all_values = record.split(',')
            
            
            inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
            
            targets = np.zeros(output_nodes) + 0.01
            
            targets[int(all_values[0])] = 0.99
            n.train(inputs, targets)
            
            inputs_plusx_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), 10, cval=0.01, order=1, reshape=False)
            n.train(inputs_plusx_img.reshape(784), targets)
            # rotated clockwise by x degrees
            inputs_minusx_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), -10, cval=0.01, order=1, reshape=False)
            n.train(inputs_minusx_img.reshape(784), targets)
            
            

In [18]:
test_data_file = open("mnist_dataset/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [19]:
scorecard = []


In [20]:
for record in test_data_list:
    
    all_values = record.split(',')
    
    correct_label = int(all_values[0])
    
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    
    outputs = n.query(inputs)
    
    label = np.argmax(outputs)
    
    if(label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0) 

In [21]:
scorecard_array = np.asarray(scorecard)

print("Performance = ", scorecard_array.sum() / scorecard_array.size)

Performance =  0.9703
